In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [68]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [69]:
df=pd.read_csv('/content/UPI__DATA__SET.csv')
df.head(6)

,Transaction_ID,Date,Time,Merchant_ID,Customer_ID,Device_ID,Transaction_Type,Payment_Gateway,Transaction_City,Transaction_State,IP_Address,Transaction_Status,Device_OS,Transaction_Frequency,Merchant_Category,Transaction_Channel,Transaction_Amount_Deviation,Days_Since_Last_Transaction,amount,fraud
0,T00022452,20-04-2023,04:50:22 AM,f65a902b-2396-40cc-9593-97e103f1bc15,89aaeceb-21f5-46c8-9de3-89dde6a10a75,c1e0deb4-7c97-4178-a838-38f4a2f0b57c,Refund,SamplePay,Durgapur,Chhattisgarh,140.213.7.48,Completed,MacOS,1,Brand Vouchers and OTT,In-store,25.02,5,396.62,1
1,T00032233,08-08-2023,08:09:21 AM,d4a5efcb-4eb6-4d3a-8132-07bb3e6e13a4,8a8962f9-e84a-4573-ab0b-187311978a21,62e14f64-b0ba-4284-889a-51ac05baf33e,Bank Transfer,SamplePay,Rajpur Sonarpur,Himachal Pradesh,184.108.177.45,Pending,Windows,30,Home delivery,Mobile,-36.64,20,121.94,1
2,T00037364,25-06-2023,07:49:01 AM,759ad138-9473-4729-8699-3d72c7ffb983,c3f53ce6-e305-4460-a71d-93bde26043ab,4281c542-ac77-4269-b4bb-1de93ac12677,Bank Transfer,Other,New Delhi,Himachal Pradesh,16.106.248.163,Failed,Android,2,Utilities,Online,44.19,22,106.69,1
3,T00015924,28-01-2024,05:44:12 AM,d8f561e4-bded-4ef0-bcd8-5494b2e31a94,9f0c5613-7d4c-4454-bee1-c47fbba406a1,058b6488-2469-42f2-bc7f-707921d35cad,Subscription,UPI Pay,Bharatpur,Chhattisgarh,65.245.160.212,Failed,Android,0,Purchases,Online,-54.34,28,3611.11,1
4,T00021805,21-06-2023,09:40:59 AM,26fdd7a1-8537-4dfe-bcf7-f5a127b36682,bf9aa6d5-7d3d-43f9-8631-31336720a383,068ff12c-127a-4cfb-9899-ddde37060b28,Investment,Dummy Bank,Sagar,Mizoram,33.172.152.38,Completed,MacOS,1,Other,Online,12.38,25,374.89,1
5,T00047124,28-08-2023,10:09:28 AM,da557bc8-c1af-464b-924f-1208adbf71e8,213a302c-3fea-4e55-ab11-de8623e2bafa,a897cf08-32e4-4182-ae3a-cfe6d3d109b2,Purchase,SamplePay,Bhiwani,Assam,131.232.177.62,Completed,iOS,20,More Services,Mobile,-21.99,7,186.83,1


In [70]:
x=df.drop('fraud',axis='columns')
y=df['fraud']

In [71]:
y

,fraud
0,1
1,1
2,1
3,1
4,1
...,...
6995,1
6996,0
6997,1
6998,1


In [72]:
simple=SimpleImputer()

In [73]:
df.dtypes

,0
Transaction_ID,object
Date,object
Time,object
Merchant_ID,object
Customer_ID,object
Device_ID,object
Transaction_Type,object
Payment_Gateway,object
Transaction_City,object
Transaction_State,object


In [74]:
categorical_features=df.select_dtypes(include=['object']).columns.tolist()

In [75]:
categorical_features

['Transaction_ID',
 'Date',
 'Time',
 'Merchant_ID',
 'Customer_ID',
 'Device_ID',
 'Transaction_Type',
 'Payment_Gateway',
 'Transaction_City',
 'Transaction_State',
 'IP_Address',
 'Transaction_Status',
 'Device_OS',
 'Merchant_Category',
 'Transaction_Channel']

In [76]:
numeric_features=df.select_dtypes(include=['int64','float64']).columns.tolist()

In [77]:
numeric_features

['Transaction_Frequency',
 'Transaction_Amount_Deviation',
 'Days_Since_Last_Transaction',
 'amount',
 'fraud']

In [78]:
categorical_processor=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('onehot',OneHotEncoder())
    ]
)

In [79]:
categorical_processor

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('onehot', OneHotEncoder())])

In [80]:
numeric_processor=Pipeline(
    steps=[
        ('imputer', SimpleImputer(missing_values=np.nan,strategy='mean')),
        ('scaler', StandardScaler())
    ]
)

In [81]:
numeric_processor

Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler())])

In [82]:
preprocessor=ColumnTransformer(
    transformers=[
        ('cat',categorical_processor,categorical_features),
        ('num',numeric_processor,[feature for feature in numeric_features if feature != 'fraud'])
    ]
)
preprocessor

ColumnTransformer(transformers=[('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot', OneHotEncoder())]),
                                 ['Transaction_ID', 'Date', 'Time',
                                  'Merchant_ID', 'Customer_ID', 'Device_ID',
                                  'Transaction_Type', 'Payment_Gateway',
                                  'Transaction_City', 'Transaction_State',
                                  'IP_Address', 'Transaction_Status',
                                  'Device_OS', 'Merchant_Category',
                                  'Transaction_Channel']),
                                ('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['Transaction_Frequency',
                                  'Transaction_Amount_Deviation',
                                  'Days_Since_Last_Transaction', 'amount'])])

<h1>Random forest</h1>

In [83]:
model=Pipeline(
    steps=[
        ('preprocessor',preprocessor),
        ('classifier',RandomForestClassifier(n_estimators=100,random_state=42))
    ]
)
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder())]),
                                                  ['Transaction_ID', 'Date',
                                                   'Time', 'Merchant_ID',
                                                   'Customer_ID', 'Device_ID',
                                                   'Transaction_Type',
                                                   'Payment_Gateway',
                                                   'Transaction_City',
                                                   'Transaction_State',
                                                   'IP_Address',
                                                   'Transaction_Status',
                                                   'Device_OS',
                                                   'Merchant_Category',
                                                   'Transaction_Channel']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Transaction_Frequency',
                                                   'Transaction_Amount_Deviation',
                                                   'Days_Since_Last_Transaction',
                                                   'amount'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [109]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
model.fit(x_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder())]),
                                                  ['Transaction_ID', 'Date',
                                                   'Time', 'Merchant_ID',
                                                   'Customer_ID', 'Device_ID',
                                                   'Transaction_Type',
                                                   'Payment_Gateway',
                                                   'Transaction_City',
                                                   'Transaction_State',
                                                   'IP_Address',
                                                   'Transaction_Status',
                                                   'Device_OS',
                                                   'Merchant_Category',
                                                   'Transaction_Channel']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Transaction_Frequency',
                                                   'Transaction_Amount_Deviation',
                                                   'Days_Since_Last_Transaction',
                                                   'amount'])])),
                ('classifier', LogisticRegression())])

In [110]:
score=model.score(x_test,y_test)

In [111]:
print("Model Accuracy:", round(score, 3))

Model Accuracy: 1.0


In [112]:
model.predict(x_test)

array([0, 0, 0, ..., 0, 0, 0])

<h1>SVM</h1>

In [113]:
model=Pipeline(
    steps=[
        ('preprocessor',preprocessor),
        ('classifier',SVC())
    ]
)
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder())]),
                                                  ['Transaction_ID', 'Date',
                                                   'Time', 'Merchant_ID',
                                                   'Customer_ID', 'Device_ID',
                                                   'Transaction_Type',
                                                   'Payment_Gateway',
                                                   'Transaction_City',
                                                   'Transaction_State',
                                                   'IP_Address',
                                                   'Transaction_Status',
                                                   'Device_OS',
                                                   'Merchant_Category',
                                                   'Transaction_Channel']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Transaction_Frequency',
                                                   'Transaction_Amount_Deviation',
                                                   'Days_Since_Last_Transaction',
                                                   'amount'])])),
                ('classifier', SVC())])

In [114]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
model.fit(x_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder())]),
                                                  ['Transaction_ID', 'Date',
                                                   'Time', 'Merchant_ID',
                                                   'Customer_ID', 'Device_ID',
                                                   'Transaction_Type',
                                                   'Payment_Gateway',
                                                   'Transaction_City',
                                                   'Transaction_State',
                                                   'IP_Address',
                                                   'Transaction_Status',
                                                   'Device_OS',
                                                   'Merchant_Category',
                                                   'Transaction_Channel']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Transaction_Frequency',
                                                   'Transaction_Amount_Deviation',
                                                   'Days_Since_Last_Transaction',
                                                   'amount'])])),
                ('classifier', SVC())])

In [115]:
score=model.score(x_test,y_test)

In [116]:
print("Model Accuracy:", round(score, 3))

Model Accuracy: 1.0


In [117]:
model.predict(x_test)

array([0, 0, 0, ..., 0, 0, 0])

<h1>KNN</h1>

In [118]:
model=Pipeline(
    steps=[
        ('preprocessor',preprocessor),
        ('classifier',KNeighborsClassifier())
    ]
)
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder())]),
                                                  ['Transaction_ID', 'Date',
                                                   'Time', 'Merchant_ID',
                                                   'Customer_ID', 'Device_ID',
                                                   'Transaction_Type',
                                                   'Payment_Gateway',
                                                   'Transaction_City',
                                                   'Transaction_State',
                                                   'IP_Address',
                                                   'Transaction_Status',
                                                   'Device_OS',
                                                   'Merchant_Category',
                                                   'Transaction_Channel']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Transaction_Frequency',
                                                   'Transaction_Amount_Deviation',
                                                   'Days_Since_Last_Transaction',
                                                   'amount'])])),
                ('classifier', KNeighborsClassifier())])

In [119]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
model.fit(x_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder())]),
                                                  ['Transaction_ID', 'Date',
                                                   'Time', 'Merchant_ID',
                                                   'Customer_ID', 'Device_ID',
                                                   'Transaction_Type',
                                                   'Payment_Gateway',
                                                   'Transaction_City',
                                                   'Transaction_State',
                                                   'IP_Address',
                                                   'Transaction_Status',
                                                   'Device_OS',
                                                   'Merchant_Category',
                                                   'Transaction_Channel']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Transaction_Frequency',
                                                   'Transaction_Amount_Deviation',
                                                   'Days_Since_Last_Transaction',
                                                   'amount'])])),
                ('classifier', KNeighborsClassifier())])

In [120]:
score=model.score(x_test,y_test)

In [121]:
print("Model Accuracy:", round(score, 3))

Model Accuracy: 0.997


In [122]:
model.predict(x_test)

array([0, 0, 0, ..., 0, 0, 0])

<h1>Linear Regression</h1>

In [123]:
model=Pipeline(
    steps=[
        ('preprocessor',preprocessor),
        ('classifier',LogisticRegression())
    ]
)
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder())]),
                                                  ['Transaction_ID', 'Date',
                                                   'Time', 'Merchant_ID',
                                                   'Customer_ID', 'Device_ID',
                                                   'Transaction_Type',
                                                   'Payment_Gateway',
                                                   'Transaction_City',
                                                   'Transaction_State',
                                                   'IP_Address',
                                                   'Transaction_Status',
                                                   'Device_OS',
                                                   'Merchant_Category',
                                                   'Transaction_Channel']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Transaction_Frequency',
                                                   'Transaction_Amount_Deviation',
                                                   'Days_Since_Last_Transaction',
                                                   'amount'])])),
                ('classifier', LogisticRegression())])

In [124]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
model.fit(x_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder())]),
                                                  ['Transaction_ID', 'Date',
                                                   'Time', 'Merchant_ID',
                                                   'Customer_ID', 'Device_ID',
                                                   'Transaction_Type',
                                                   'Payment_Gateway',
                                                   'Transaction_City',
                                                   'Transaction_State',
                                                   'IP_Address',
                                                   'Transaction_Status',
                                                   'Device_OS',
                                                   'Merchant_Category',
                                                   'Transaction_Channel']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Transaction_Frequency',
                                                   'Transaction_Amount_Deviation',
                                                   'Days_Since_Last_Transaction',
                                                   'amount'])])),
                ('classifier', LogisticRegression())])

In [125]:
score=model.score(x_test,y_test)

In [126]:
print("Model Accuracy:", round(score, 3))

Model Accuracy: 1.0


In [127]:
model.predict(x_test)

array([0, 0, 0, ..., 0, 0, 0])